En el siguiente notebook vamos a trabajar con PySpark. En él, vamos a usar las transformaciones y las acciones para analizar un pequeño conjunto de datos.

El primer paso que debemos dar es conectar Google Colab con nuestro Google Drive. Para ello, lanzaremos el siguiente trozo de código.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


Una vez hemos conectado Google Drive, pasamos a instalar Apache Spark en nuestro notebook. En concreto, vamos a trabajar con la versión 3.5.0.

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q http://apache.mirrors.pair.com/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [5]:
!ls


drive  sample_data  spark-3.5.0-bin-hadoop3.tgz


In [6]:
!tar xf spark-3.5.0-bin-hadoop3.tgz

Además de la instalación habitual, hay un paso más que debemos dar, y es la instalación de la librería *findspark*. Esta librería nos permitirá encontrar la instalación de Apache Spark en nuestro sistema.

Establecemos también un par de variables de entorno.

In [7]:
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [9]:
import findspark
findspark.init()

In [10]:
findspark.find()

'/content/spark-3.5.0-bin-hadoop3'

Una vez tenemos instalado nuestro framework, es momento de crear nuestra variable SparkSession, de la cual parten la mayoría de funcionalidades de SparkSQL

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Pyspark_SQL")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [12]:
spark

### Lectura de datos

Ahora sí, podemos comenzar a trabajar con SparkSQL. Comenzaremos leyendo nuestro conjunto de datos, el cual contiene información de valoraciones de restaurantes.

In [13]:
!pwd

/content


In [14]:
import pandas as pd

data_path = '/content/drive/MyDrive/TokioSchool/'

data = spark.read.options(inferSchema='True',delimiter=',', header=True).csv (data_path + 'CSV_stocks_2021.csv')


In [15]:
data

DataFrame[ticker: string, open: double, high: double, low: double, close: double, volume: int, dividends: double, stock splits: int, date: date, ccy: string]

### Análisis estadísico

In [16]:
data.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- dividends: double (nullable = true)
 |-- stock splits: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- ccy: string (nullable = true)



Obtención de datos

*   a) Filtraremos las filas que correspondan con el valor ‘HON’ de la columna ‘ticker’




In [24]:
# Filtraremos las filas que correspondan con el valor ‘HON’ de la columna ‘ticker’
data.filter(data.ticker == 'HON').show()


+------+------------------+------------------+------------------+------------------+-------+---------+------------+----------+---+
|ticker|              open|              high|               low|             close| volume|dividends|stock splits|      date|ccy|
+------+------------------+------------------+------------------+------------------+-------+---------+------------+----------+---+
|   HON|207.45501408335315|209.42141232111007|206.14735745495346|209.12644958496094|1406400|      0.0|           0|2020-12-31|USD|
|   HON|209.26409612607986|209.43123817541044| 202.8929700588177| 204.4562530517578|2328900|      0.0|           0|2021-01-04|USD|
|   HON|203.50255708426138| 206.6586329757187|203.50255708426138|204.95770263671875|2172100|      0.0|           0|2021-01-05|USD|
|   HON|205.93106042557045|210.38495140642195|205.71475541084416|   208.69384765625|2747900|      0.0|           0|2021-01-06|USD|
|   HON|209.31327369732284|210.41445198270796|207.25839100319203|209.03797912597656

*   b) Extraeremos la columna ‘open’ y la convertiremos en un array unidimensional **usando** Numpy.

In [44]:
# importamos la biblioteca de numpy como np
import numpy as np

In [52]:
# transformamos un PySpark dataframe a Pandas para poder utilizar Python
pandas_data = data.toPandas()

In [54]:
# extraemos la columna open y la convertimos en un array unidimensional
data_array = np.array(pandas_data['open']).ravel()
print(data_array)

[207.45501408 209.26409613 203.50255708 ...  28.22999954  28.28000069
  28.17000008]


*   c) Dividiremos el array en dos para comparar dos periodos y determinar si se ha producido un cambio significativo.

In [65]:
# dividimos el array en 2 para poder comparar dos periodos
data_array_split = np.array_split(data_array, 2)
data_array_0 = data_array_split[0].astype(float)
data_mean_0 = np.mean(data_array_0)
data_array_1 = data_array_split[1].astype(float)
data_mean_1 = np.mean(data_array_1)
# mostramos los resultados
print(data_array_0)
print(data_array_1)

[207.45501408 209.26409613 203.50255708 ...  38.53870413  39.29360605
  40.03882752]
[38.71291254 39.63235014 39.81623245 ... 28.22999954 28.28000069
 28.17000008]


In [66]:
# mostramos las medias
print(data_mean_0)
print(data_mean_1)

406.42411510557037
149.69845340250652


Según los datos obtenidos, sí hay un cambio significativo

* d) Aplicaremos la prueba estadística t de Student y proporcionaremos el valor obtenido.**texto en negrita**

In [72]:
# importo biblitoeca scipy para métodos matemáticos
import scipy
print(scipy.__version__)

1.11.4


In [73]:
# aplicamos la t de student
scipy.stats.ttest_ind(data_array_0, data_array_1, equal_var=False)

TtestResult(statistic=17.217002035969895, pvalue=1.7711279373092314e-63, df=3058.1749213647395)

pvalor muy cercano a 0, por lo que se rechaza la hipóteis nula H0 y se acepta la hipóteis alternativa H1